# Deepersense: sim data generator

## imports

In [1]:
import os
# import sys
import rospy
import rosbag
import rospkg
import glob

In [2]:
from simulation_controller import SimulationController
from scene_generator import SceneGenerator, ColorMap
from launchfile_maker import LaunchFileMaker

## file manager

### cwd

In [3]:
cwd = os.getcwd()

### package path

In [4]:
rospack = rospkg.RosPack()
package_path = rospack.get_path('stonefish_scene_generator')

### iterate files in folder

In [5]:
all_files_path = os.path.join(package_path,'*')
for file_name in glob.glob(all_files_path):
    print(file_name)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch
/home/zeged/catkin_ws/src/stonefish_scene_generator/src
/home/zeged/catkin_ws/src/stonefish_scene_generator/package.xml
/home/zeged/catkin_ws/src/stonefish_scene_generator/data
/home/zeged/catkin_ws/src/stonefish_scene_generator/CMakeLists.txt


### create data folder

In [6]:
def create_folder(path):
    dir_exists = os.path.isdir(path)
    if not dir_exists:
        os.mkdir(path)
        print('data folder created: {}'.format(path))

In [7]:
data_folder_path = os.path.join(package_path, 'data')
create_folder(data_folder_path)

## generate stonefish scene

## make a scene

In [35]:
# def make_a_scene(category, main_subject):
#     scene = None
#     title = category + '_' + main_subject
#     seabed_depth = 15
#     gen = SceneGenerator(seabed_depth)
#     gen.add_empty_frame(ColorMap.jet)
#     if main_subject == 'box':
#         gen.add_boxes()
#     else:
#         gen.add_reefs()
#     scene = gen.generate()
#     return scene

## categories

In [36]:
# categories = []
# categories.append('basic')
# categories.append('multiObect')
# shapes = []
# shapes.append('box')
# shapes.append('sphere')
# shapes.append('cylindar')
# reefs = ['2019', '2020']
# for reef in reefs:
#     shapes.append(reef)

# test = [categories, shapes]
# import itertools
# for category, main_subject in list(itertools.product(*test)):
#     print(category, main_subject)
#     scene = make_a_scene(category, main_subject)

('basic', 'box')
('basic', 'sphere')
('basic', 'cylindar')
('basic', '2019')
('basic', '2020')
('multiObect', 'box')
('multiObect', 'sphere')
('multiObect', 'cylindar')
('multiObect', '2019')
('multiObect', '2020')


### generate scene

In [8]:
title = 'basic'
seabed_depth = 15
gen = SceneGenerator(seabed_depth)
gen.add_empty_frame(ColorMap.jet)
gen.add_reefs()
scene = gen.generate()
# print(scene)

### save scene to .scn file

In [9]:
scene_file_path = os.path.join(data_folder_path, title + '.scn')
f = open(scene_file_path, "w")
f.write(scene)
f.close()

## Simulation

### get launch_file_path

In [10]:
launch_file_name = 'auto_generated.launch'
launch_file_path = os.path.join(package_path, 'launch', launch_file_name)
print(launch_file_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch/auto_generated.launch


### edit/make launch file

In [11]:
rate = 300.0
resolution = (1200, 800)
bagfile_path = os.path.join(data_folder_path, title + '.bag')
maker = LaunchFileMaker(scene_file_path, rate, resolution, bagfile_path)
maker.save_to_file(launch_file_path)

### setup simulation

In [12]:
controller = SimulationController(launch_file_path)

... logging to /home/zeged/.ros/log/138a2ef6-28c2-11ec-aa88-107b448dffed/roslaunch-zeged-System-Product-Name-3772.log


### launch simulation

In [13]:
controller.run()
controller.wait_for_ros()

started roslaunch server http://zeged-System-Product-Name:46283/

SUMMARY

PARAMETERS
 * /rosdistro: melodic
 * /rosversion: 1.14.11

NODES
  /
    rosbag_record_cam (rosbag/record)
    rviz (rviz/rviz)
    stonefish_simulator (stonefish_ros/parsed_simulator)

auto-starting new master
process[master]: started with pid [3819]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 138a2ef6-28c2-11ec-aa88-107b448dffed
process[rosout-1]: started with pid [3837]
started core service [/rosout]
process[stonefish_simulator-2]: started with pid [3840]
process[rviz-3]: started with pid [3845]
process[rosbag_record_cam-4]: started with pid [3846]
[INFO] [1633757456.833538]: started
[INFO] [1633757456.833895]: waiting for topic msg
waiting for message
wait is over
rospy.get_time() - self.timer > 15
(1633757485.684022, 1633757470.683414)
[INFO] [1633757485.684169]: timeout: shutting down
[rosbag_record_cam-4] killing on exit
[rviz-3] killing on exit
[stonefish_simulator-2] killing on exit
[rosout

## Bag

### open bag file

In [14]:
import rospy
from collections import defaultdict
import cv2
import numpy as np
from cv_bridge import CvBridge

bag = rosbag.Bag(bagfile, 'r')

## open bag file

In [15]:
bag = rosbag.Bag(bagfile_path, 'r')

## create a folder for images

In [16]:
image_folder_path = bagfile_path.split('.')[0]
print(image_folder_path)
create_folder(image_folder_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic
data folder created: /home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic


In [17]:
def imshow(win, img):
    cv2.imshow(win, img)
    cv2.waitKey(10)
     

In [18]:
def process_image(msg, topic):
    bridge = CvBridge()
    encoding = 'bgr8'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [19]:
def process_depth(msg, topic):
    bridge = CvBridge()
    encoding = '32FC1'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [20]:
def normaliaze_depth(msg):
    cv_image_array = np.array(msg, dtype = np.dtype('f8'))
    slice1Copy = cv_image_norm = cv2.normalize(cv_image_array, cv_image_array, 0, 1, cv2.NORM_MINMAX)
    slice1Copy = np.uint8(slice1Copy*255)
    return(slice1Copy)

In [21]:
def save_image(img, filename):
    # print(file_name)
    cv2.imwrite(filename, img)

In [22]:
iterator = bag.read_messages()
counter = defaultdict(lambda: 0)
image_num = 0
for topic, msg, time in iterator:
    msg_time = rospy.Time.to_sec(time) # .to_time(time)
    msg_type = str(type(msg)).split('__')[1].split("'")[0]
    print(msg_time, msg_type, topic)
    counter[topic]+=1
    if msg_type == 'CameraInfo':
        continue
    image_num += 1

    if 'image_color' in topic or 'display' in topic:
        bridged_image = process_image(msg, str(topic))
        imshow(topic, bridged_image)
        file_name = os.path.join(image_folder_path, str(image_num) + '.tif')
        save_image(bridged_image, file_name)



    elif 'image_depth2/image_depth' in topic:
        bridged_image = process_depth(msg, str(topic))
        norm_image = normaliaze_depth(bridged_image)
        imshow(topic, norm_image)
        file_name = os.path.join(image_folder_path, str(image_num) + '.tif')
        save_image(norm_image, file_name)
    else:
        print(topic)

cv2.destroyAllWindows()

(1633757470.6486912, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633757470.680808, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633757470.6824439, 'Image', '/camera2/image_raw/image_color')
(1633757470.7029753, 'CameraInfo', '/image_depth2/camera_info')
(1633757470.7046473, 'Image', '/image_depth2/image_depth')
(1633757470.7517152, 'CameraInfo', '/image_depth2/camera_info')
(1633757470.7529461, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633757470.7532976, 'Image', '/image_depth2/image_depth')
(1633757470.7542408, 'Image', '/camera2/image_raw/image_color')
(1633757470.7803667, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633757470.8302991, 'CameraInfo', '/image_depth2/camera_info')
(1633757470.8309119, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633757470.8317757, 'Image', '/image_depth2/image_depth')
(1633757470.8320181, 'Image', '/camera2/image_raw/image_color')
(1633757470.8738651, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633757470.9233935, 'C

In [23]:
print(counter)

defaultdict(<function <lambda> at 0x7f2354e40bd0>, {'/sparus2/Blue_view_M900_FLS2/display': 175, '/camera2/image_raw/image_color': 249, '/image_depth2/camera_info': 182, '/camera2/image_raw/camera_info': 249, '/image_depth2/image_depth': 182})
